<a href="https://colab.research.google.com/github/harshavardhinisri/CS-6320-NLP/blob/main/nlp_a2_q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
import math
import random
import os
import time
from tqdm import tqdm
import json
from argparse import ArgumentParser

In [ ]:
unk = '<UNK>'

In [ ]:
# The FFNN class defines a simple feedforward neural network (FFNN) with
# one hidden layer in PyTorch. It takes an input vector and first applies a
# linear transformation (W1) followed by a ReLU activation function to generate
# a hidden representation. This hidden representation is then transformed by another
# linear layer (W2) and passed through a softmax function to produce a predicted output
# vector, which is used for classification tasks. The network also includes a method
# (compute_Loss) to calculate the negative log-likelihood loss for training.

class FFNN(nn.Module):
    def __init__(self, input_dim, h):
        super(FFNN, self).__init__()
        self.h = h
        self.W1 = nn.Linear(input_dim, h)
        self.activation = nn.ReLU()
        self.output_dim = 5
        self.W2 = nn.Linear(h, self.output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        self.loss = nn.NLLLoss()

    def compute_Loss(self, predicted_vector, gold_label):
        return self.loss(predicted_vector, gold_label)

    def forward(self, input_vector):
        hidden_representation = self.activation(self.W1(input_vector))
        output_representation = self.W2(hidden_representation)
        predicted_vector = self.softmax(output_representation)
        return predicted_vector

In [ ]:
def make_vocab(data):
    vocab = set()
    for document, _ in data:
        for word in document:
            vocab.add(word)
    return vocab

In [ ]:
def make_indices(vocab):
    vocab_list = sorted(vocab)
    vocab_list.append(unk)
    word2index = {}
    index2word = {}
    for index, word in enumerate(vocab_list):
        word2index[word] = index
        index2word[index] = word
    vocab.add(unk)
    return vocab, word2index, index2word

In [ ]:
def convert_to_vector_representation(data, word2index):
    vectorized_data = []
    for document, y in data:
        vector = torch.zeros(len(word2index))
        for word in document:
            index = word2index.get(word, word2index[unk])
            vector[index] += 1
        vectorized_data.append((vector, y))
    return vectorized_data

In [ ]:
def load_data(train_data, val_data):
    with open(train_data) as training_f:
        training = json.load(training_f)
    with open(val_data) as valid_f:
        validation = json.load(valid_f)

    tra = []
    val = []
    for elt in training:
        tra.append((elt["text"].split(), int(elt["stars"]) - 1))
    for elt in validation:
        val.append((elt["text"].split(), int(elt["stars"]) - 1))

    return tra, val

In [ ]:
class Args:
    hidden_dim = 128
    epochs = 10
    train_data = 'training.json'
    val_data = 'validation.json'
    test_data = 'test.json'
    do_train = True

args = Args()

random.seed(42)
torch.manual_seed(42)

In [ ]:
# the training data is shuffled and processed in minibatches, with loss
# computed and gradients backpropagated to update the model's parameters.
# Similarly, the model's performance is validated on the validation data,
# with both training and validation accuracies printed after each epoch to
# monitor progress. The script also includes timing for each epoch to track
# the training and validation duration

print("========== Loading data ==========")
train_data, valid_data = load_data(args.train_data, args.val_data)
vocab = make_vocab(train_data)
vocab, word2index, index2word = make_indices(vocab)

print("========== Vectorizing data ==========")
train_data = convert_to_vector_representation(train_data, word2index)
valid_data = convert_to_vector_representation(valid_data, word2index)

model = FFNN(input_dim=len(vocab), h=args.hidden_dim)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

print("========== Training for {} epochs ==========".format(args.epochs))
for epoch in range(args.epochs):
    model.train()
    optimizer.zero_grad()
    loss = None
    correct = 0
    total = 0
    start_time = time.time()
    print("Training started for epoch {}".format(epoch + 1))
    random.shuffle(train_data)
    minibatch_size = 64  # Increase the batch size
    N = len(train_data)
    for minibatch_index in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        loss = None
        for example_index in range(minibatch_size):
            input_vector, gold_label = train_data[minibatch_index * minibatch_size + example_index]
            predicted_vector = model(input_vector.float().unsqueeze(0))
            predicted_label = torch.argmax(predicted_vector)
            correct += int(predicted_label == gold_label)
            total += 1
            example_loss = model.compute_Loss(predicted_vector.view(1, -1), torch.tensor([gold_label]))
            if loss is None:
                loss = example_loss
            else:
                loss += example_loss
        loss = loss / minibatch_size
        loss.backward()
        optimizer.step()
    print("Training completed for epoch {}".format(epoch + 1))
    print("Training accuracy for epoch {}: {}".format(epoch + 1, correct / total))
    print("Training time for this epoch: {}".format(time.time() - start_time))

    loss = None
    correct = 0
    total = 0
    start_time = time.time()
    print("Validation started for epoch {}".format(epoch + 1))
    minibatch_size = 16
    N = len(valid_data)

    # Validation code (similar to training)
    for minibatch_index in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        loss = None
        for example_index in range(minibatch_size):
            input_vector, gold_label = valid_data[minibatch_index * minibatch_size + example_index]
            predicted_vector = model(input_vector.float().unsqueeze(0))
            predicted_label = torch.argmax(predicted_vector)
            correct += int(predicted_label == gold_label)
            total += 1
            example_loss = model.compute_Loss(predicted_vector.view(1, -1), torch.tensor([gold_label]))
            if loss is None:
                loss = example_loss
            else:
                loss += example_loss
    print("Validation completed for epoch {}".format(epoch + 1))
    print("Validation accuracy for epoch {}: {}".format(epoch + 1, correct / total))
    print("Validation time for this epoch: {}".format(time.time() - start_time))


========== Loading data ==========
========== Vectorizing data ==========
========== Training for 10 epochs ==========
Training started for epoch 1


100%|██████████| 250/250 [02:56<00:00,  1.42it/s]


Training completed for epoch 1
Training accuracy for epoch 1: 0.412875
Training time for this epoch: 176.40744638442993
Validation started for epoch 1


100%|██████████| 50/50 [00:00<00:00, 61.55it/s]


Validation completed for epoch 1
Validation accuracy for epoch 1: 0.405
Validation time for this epoch: 0.8166606426239014
Training started for epoch 2


...

Validation completed for epoch 9
Validation accuracy for epoch 9: 0.575
Validation time for this epoch: 0.8334078788757324
Training started for epoch 10


100%|██████████| 250/250 [03:07<00:00,  1.33it/s]


Training completed for epoch 10
Training accuracy for epoch 10: 0.7400625
Training time for this epoch: 187.6691815853119
Validation started for epoch 10


100%|██████████| 50/50 [00:00<00:00, 60.03it/s]

Validation completed for epoch 10
Validation accuracy for epoch 10: 0.55625
Validation time for this epoch: 0.8382613658905029


In [ ]:
def load_test_data(test_data):
    with open(test_data) as test_f:
        test = json.load(test_f)
    test_data = [(elt["text"].split(), int(elt["stars"]-1)) for elt in test]
    return test_data

# Assuming Args class is defined as before
args = Args()

# Load and process test data
test_data = load_test_data(args.test_data)
test_data = convert_to_vector_representation(test_data, word2index)

# Testing
model.eval()
total_correct, total = 0, 0
print("Testing started")
for input_vector, gold_label in test_data:
    input_vector = input_vector.float().unsqueeze(0)  # Adjusting for batch dimension
    predicted_vector = model(input_vector)
    predicted_label = torch.argmax(predicted_vector)
    total_correct += int(predicted_label == gold_label)
    total += 1

test_accuracy = (total_correct / total)*100
print(f"Testing Accuracy: {test_accuracy:.2f}%")



Testing started
Testing Accuracy: 55.62%


In [ ]:
# This model features a more complex architecture with five hidden layers,
# each with different dimensions, where the size of intermediate layers is increased
# and then decreased, forming a sort of pyramid structure. The layers are
# implemented as linear transformations (W1 to W6) followed by
# ReLU activation functions.

import torch.nn as nn

class DeepFFNN(nn.Module):
    def __init__(self, input_dim, h):
        super(DeepFFNN, self).__init__()

        # Define the hidden layers and their dimensions
        self.h1 = h
        self.h2 = int(h*1.5)  # Adjust the size as needed
        self.h3 = int(h*2)  # Adjust the size as needed
        self.h4 = int(h*1.5)  # Adjust the size as needed
        self.h5 = h  # Adjust the size as needed, remove if not needed

        # Define the linear transformations for each layer
        self.W1 = nn.Linear(input_dim, self.h1)
        self.W2 = nn.Linear(self.h1, self.h2)
        self.W3 = nn.Linear(self.h2, self.h3)
        self.W4 = nn.Linear(self.h3, self.h4)
        self.W5 = nn.Linear(self.h4, self.h5)  # Remove if not needed
        self.W6 = nn.Linear(self.h5, 5)  # Change this to W5 if the fifth layer is not needed

        # Define the activation function
        self.activation = nn.ReLU()

        # Define the output functions
        self.softmax = nn.LogSoftmax(dim=1)
        self.loss = nn.NLLLoss()

    def compute_Loss(self, predicted_vector, gold_label):
        return self.loss(predicted_vector, gold_label)

    def forward(self, input_vector):
        # Forward pass through each layer with activation
        x = self.activation(self.W1(input_vector))
        x = self.activation(self.W2(x))
        x = self.activation(self.W3(x))
        x = self.activation(self.W4(x))
        x = self.activation(self.W5(x))  # Remove if not needed
        predicted_vector = self.softmax(self.W6(x))  # Change this to W5 if the fifth layer is not needed
        return predicted_vector


In [ ]:
print("========== Loading data ==========")
train_data, valid_data = load_data(args.train_data, args.val_data)
vocab = make_vocab(train_data)
vocab, word2index, index2word = make_indices(vocab)

print("========== Vectorizing data ==========")
train_data = convert_to_vector_representation(train_data, word2index)
valid_data = convert_to_vector_representation(valid_data, word2index)

model = DeepFFNN(input_dim=len(vocab), h=args.hidden_dim)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

print("========== Training for {} epochs ==========".format(args.epochs))
for epoch in range(args.epochs):
    model.train()
    total_loss, total_correct, total = 0, 0, 0
    start_time = time.time()
    print("Training started for epoch {}".format(epoch + 1))
    random.shuffle(train_data)
    minibatch_size = 64  # Increased batch size
    N = len(train_data)

    for minibatch_index in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        minibatch_loss = None
        for example_index in range(minibatch_size):
            input_vector, gold_label = train_data[minibatch_index * minibatch_size + example_index]
            input_vector = input_vector.float().unsqueeze(0)  # Adjusting for batch dimension
            predicted_vector = model(input_vector)
            predicted_label = torch.argmax(predicted_vector)
            total_correct += int(predicted_label == gold_label)
            total += 1
            example_loss = model.compute_Loss(predicted_vector.view(1, -1), torch.tensor([gold_label]))
            minibatch_loss = example_loss if minibatch_loss is None else minibatch_loss + example_loss
        minibatch_loss = minibatch_loss / minibatch_size
        total_loss += minibatch_loss.item()
        minibatch_loss.backward()
        optimizer.step()

    print("Training completed for epoch {}".format(epoch + 1))
    print("Training accuracy for epoch {}: {:.2f}%".format(epoch + 1, (total_correct / total) * 100))
    print("Training time for this epoch: {:.2f} seconds".format(time.time() - start_time))

    # Validation
    model.eval()
    total_correct, total = 0, 0
    start_time = time.time()
    print("Validation started for epoch {}".format(epoch + 1))
    minibatch_size = 16  # Validation batch size
    N = len(valid_data)

    for minibatch_index in tqdm(range(N // minibatch_size)):
        minibatch_loss = None
        for example_index in range(minibatch_size):
            input_vector, gold_label = valid_data[minibatch_index * minibatch_size + example_index]
            input_vector = input_vector.float().unsqueeze(0)  # Adjusting for batch dimension
            predicted_vector = model(input_vector)
            predicted_label = torch.argmax(predicted_vector)
            total_correct += int(predicted_label == gold_label)
            total += 1
            example_loss = model.compute_Loss(predicted_vector.view(1, -1), torch.tensor([gold_label]))
            minibatch_loss = example_loss if minibatch_loss is None else minibatch_loss + example_loss

    print("Validation completed for epoch {}".format(epoch + 1))
    print("Validation accuracy for epoch {}: {:.2f}%".format(epoch + 1, (total_correct / total) * 100))
    print("Validation time for this epoch: {:.2f} seconds".format(time.time() - start_time))



========== Loading data ==========
========== Vectorizing data ==========
========== Training for 10 epochs ==========
Training started for epoch 1


100%|██████████| 250/250 [03:08<00:00,  1.33it/s]


Training completed for epoch 1
Training accuracy for epoch 1: 20.99%
Training time for this epoch: 188.17 seconds
Validation started for epoch 1


100%|██████████| 50/50 [00:00<00:00, 52.58it/s]


Validation completed for epoch 1
Validation accuracy for epoch 1: 24.00%
Validation time for this epoch: 0.95 seconds
Training started for epoch 2


...

Validation completed for epoch 9
Validation accuracy for epoch 9: 53.50%
Validation time for this epoch: 0.99 seconds
Training started for epoch 10


100%|██████████| 250/250 [03:21<00:00,  1.24it/s]


Training completed for epoch 10
Training accuracy for epoch 10: 70.68%
Training time for this epoch: 201.12 seconds
Validation started for epoch 10


100%|██████████| 50/50 [00:00<00:00, 53.18it/s]

Validation completed for epoch 10
Validation accuracy for epoch 10: 46.12%
Validation time for this epoch: 0.94 seconds


In [ ]:
def load_test_data(test_data):
    with open(test_data) as test_f:
        test = json.load(test_f)
    test_data = [(elt["text"].split(), int(elt["stars"]-1)) for elt in test]
    return test_data

# Assuming Args class is defined as before
args = Args()

# Load and process test data
test_data = load_test_data(args.test_data)
test_data = convert_to_vector_representation(test_data, word2index)

# Testing
model.eval()
total_correct, total = 0, 0
print("Testing started")
for input_vector, gold_label in test_data:
    input_vector = input_vector.float().unsqueeze(0)  # Adjusting for batch dimension
    predicted_vector = model(input_vector)
    predicted_label = torch.argmax(predicted_vector)
    total_correct += int(predicted_label == gold_label)
    total += 1

test_accuracy = (total_correct / total)*100
print(f"Testing Accuracy: {test_accuracy:.2f}%")



Testing started
Testing Accuracy: 57.75%


In [ ]:
class Args:
    hidden_dim = 128
    epochs = 15
    train_data = 'training.json'
    val_data = 'validation.json'
    test_data = 'test.json'
    do_train = True

args = Args()

random.seed(42)
torch.manual_seed(42)

In [ ]:
print("========== Loading data ==========")
train_data, valid_data = load_data(args.train_data, args.val_data)
vocab = make_vocab(train_data)
vocab, word2index, index2word = make_indices(vocab)

print("========== Vectorizing data ==========")
train_data = convert_to_vector_representation(train_data, word2index)
valid_data = convert_to_vector_representation(valid_data, word2index)

model = DeepFFNN(input_dim=len(vocab), h=args.hidden_dim)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

print("========== Training for {} epochs ==========".format(args.epochs))
for epoch in range(args.epochs):
    model.train()
    total_loss, total_correct, total = 0, 0, 0
    start_time = time.time()
    print("Training started for epoch {}".format(epoch + 1))
    random.shuffle(train_data)
    minibatch_size = 64  # Increased batch size
    N = len(train_data)

    for minibatch_index in tqdm(range(N // minibatch_size)):
        optimizer.zero_grad()
        minibatch_loss = None
        for example_index in range(minibatch_size):
            input_vector, gold_label = train_data[minibatch_index * minibatch_size + example_index]
            input_vector = input_vector.float().unsqueeze(0)  # Adjusting for batch dimension
            predicted_vector = model(input_vector)
            predicted_label = torch.argmax(predicted_vector)
            total_correct += int(predicted_label == gold_label)
            total += 1
            example_loss = model.compute_Loss(predicted_vector.view(1, -1), torch.tensor([gold_label]))
            minibatch_loss = example_loss if minibatch_loss is None else minibatch_loss + example_loss
        minibatch_loss = minibatch_loss / minibatch_size
        total_loss += minibatch_loss.item()
        minibatch_loss.backward()
        optimizer.step()

    print("Training completed for epoch {}".format(epoch + 1))
    print("Training accuracy for epoch {}: {:.2f}%".format(epoch + 1, (total_correct / total) * 100))
    print("Training time for this epoch: {:.2f} seconds".format(time.time() - start_time))

    # Validation
    model.eval()
    total_correct, total = 0, 0
    start_time = time.time()
    print("Validation started for epoch {}".format(epoch + 1))
    minibatch_size = 16  # Validation batch size
    N = len(valid_data)

    for minibatch_index in tqdm(range(N // minibatch_size)):
        minibatch_loss = None
        for example_index in range(minibatch_size):
            input_vector, gold_label = valid_data[minibatch_index * minibatch_size + example_index]
            input_vector = input_vector.float().unsqueeze(0)  # Adjusting for batch dimension
            predicted_vector = model(input_vector)
            predicted_label = torch.argmax(predicted_vector)
            total_correct += int(predicted_label == gold_label)
            total += 1
            example_loss = model.compute_Loss(predicted_vector.view(1, -1), torch.tensor([gold_label]))
            minibatch_loss = example_loss if minibatch_loss is None else minibatch_loss + example_loss

    print("Validation completed for epoch {}".format(epoch + 1))
    print("Validation accuracy for epoch {}: {:.2f}%".format(epoch + 1, (total_correct / total) * 100))
    print("Validation time for this epoch: {:.2f} seconds".format(time.time() - start_time))



========== Loading data ==========
========== Vectorizing data ==========
========== Training for 15 epochs ==========
Training started for epoch 1


100%|██████████| 250/250 [03:15<00:00,  1.28it/s]


Training completed for epoch 1
Training accuracy for epoch 1: 20.35%
Training time for this epoch: 196.00 seconds
Validation started for epoch 1


100%|██████████| 50/50 [00:01<00:00, 49.92it/s]


Validation completed for epoch 1
Validation accuracy for epoch 1: 0.75%
Validation time for this epoch: 1.01 seconds
Training started for epoch 2


...

Validation completed for epoch 14
Validation accuracy for epoch 14: 52.25%
Validation time for this epoch: 0.94 seconds
Training started for epoch 15


100%|██████████| 250/250 [03:24<00:00,  1.22it/s]


Training completed for epoch 15
Training accuracy for epoch 15: 81.24%
Training time for this epoch: 204.84 seconds
Validation started for epoch 15


100%|██████████| 50/50 [00:00<00:00, 51.64it/s]

Validation completed for epoch 15
Validation accuracy for epoch 15: 61.62%
Validation time for this epoch: 0.98 seconds


In [ ]:
#Epoch is set to 15 using improved FFNN. However the model tend to overfit

def load_test_data(test_data):
    with open(test_data) as test_f:
        test = json.load(test_f)
    test_data = [(elt["text"].split(), int(elt["stars"]-1)) for elt in test]
    return test_data

# Assuming Args class is defined as before
args = Args()

# Load and process test data
test_data = load_test_data(args.test_data)
test_data = convert_to_vector_representation(test_data, word2index)

# Testing
model.eval()
total_correct, total = 0, 0
print("Testing started")
for input_vector, gold_label in test_data:
    input_vector = input_vector.float().unsqueeze(0)  # Adjusting for batch dimension
    predicted_vector = model(input_vector)
    predicted_label = torch.argmax(predicted_vector)
    total_correct += int(predicted_label == gold_label)
    total += 1

test_accuracy = (total_correct / total)*100
print(f"Testing Accuracy: {test_accuracy:.2f}%")



Testing started
Testing Accuracy: 46.00%
